In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bisect
import datetime
import json
import pathlib

In [79]:
import fumbblapi
import srdata
import srpages
import srschedule
import srtime
import srtournament

In [4]:
sr_data_path = '../sr-data'
sr_pages_path = '../sr-pages'

In [16]:
srdata.load(sr_data_path)

In [6]:
TOURNAMENT = 0
TOP_TOURNAMENT = 1
GROUP = 2
NAME = 3
CLASS = 4
TITLE = 5
FIRST_SLOT_GROUP = 6
ENTER_WEEKNR = 7
EXIT_WEEKNR = 8

In [7]:
updated = srtime.now().strftime('%Y-%m-%d %H:%M')
updated

'2018-06-04 18:43'

## Add New Tournaments

In [8]:
tournament_format_to_elim = {'King': 'E', 'Knockout': 'E', 'RoundRobin': 'N', 'Swiss': 'N'}
titles = {row[5] for row in srdata.data["tournaments"] if 6 <= len(row) and row[5]}
print('Current titles: ' + ', '.join(sorted(titles)))
slots = set(srdata.data["slot"].keys())
print('Current slots: ' + ', '.join(sorted(slots)))

Current titles: BALL, BC, CC, COS, FC, FUBC, GLT, LC, UI, WO, WOC, XFL AMA, XFL CD, XFL CHA, XFL CP, XFL DE, XFL DWA, XFL ELF, XFL GOB, XFL HE, XFL HFL, XFL HUM, XFL KHE, XFL LIZ, XFL MAS, XFL NEC, XFL NOR, XFL NUR, XFL OGR, XFL ORC, XFL SKA, XFL SLA, XFL UD, XFL UW, XFL VAM, XFL WE
Current slots: FC, MA, NE, R, W


In [9]:
new_rows = []
for groupId in srdata.data["groups"]:
    group_tournaments = fumbblapi.get__group_tournaments(groupId)
    for tournament in group_tournaments:
        tournamentId = tournament["id"]
        class_parts = [None] * 4
        if tournamentId not in srdata.data["tournament"]:
            print(f'{tournament["name"].strip()} [{tournamentId}|{groupId}]')
            print('-' * 100)
            # asking for tournament name
            name = input('Name? (Hit Enter to keep current one) ').strip()
            if not name:
                name = tournament["name"].strip()
            # asking for main class and level
            while class_parts[0] not in {'MI', 'MA', 'QU'}:
                class_parts[0] = input('Main Class? (MI/MA/QU; Enter for MI) ').strip().upper()
                if not class_parts[0]:
                    class_parts[0] = 'MI'
            while True:
                class_parts[1] = input('Level? (integer; Enter for 1) ').strip()
                if not class_parts[1]:
                    class_parts[1] = '1'
                if not class_parts[1].isdecimal():
                    continue
                alloc_keys = {k for k in srdata.data["alloc"].keys() if k.startswith('/'.join(class_parts[:2]))}
                if alloc_keys:
                    break
            if class_parts[0] != 'QU':
                # asking for title
                while True:
                    title = input('Title? (Enter for none) ').strip().upper()
                    if title and title not in titles:
                        sure = None
                        while sure not in ('Y', 'N'):
                            sure = input('Are you sure to create a new title? (Y/N) ').strip().upper()
                        if sure == 'N':
                            continue
                    break
                # asking for slot
                slot = None
                while slot not in slots:
                    slot = input('Slot? (Enter for R) ').strip().upper()
                    if not slot:
                        slot = 'R'
            if class_parts[0] == 'MA':
                # asking whether to join unchained qualifiers
                qurows = [row for row in srdata.data["tournament"] if row[1] is None and row[4].startswith('QU')]
                if qurows:
                    print('Check whether the following tournaments are qualifiers of this one (Y/N): ')
                for row2 in qurows:
                    yn = None
                    while yn not in ('Y', 'N'):
                        yn = input(f' {row2[NAME]} [{row2[TOURNAMENT]}|{row2[GROUP]}]').srtip().upper()
                    if yn == 'Y':
                        row2[TOP_TOURNAMENT] = tournamentId
            # determining type
            tournament_format = srtournament.format_(groupId, tournamentId)
            class_parts[2] = tournament_format_to_elim[tournament_format]
            # determining size class
            alloc_keys = {k for k in srdata.data["alloc"].keys() if k.startswith('/'.join(class_parts[:3]))}
            size_classes = [a.split('/')[-1].split('-') for a in alloc_keys]
            size_classes = [([t[0], t[1]] if len(t)==1 else t) for t in size_classes]
            size_classes = sorted([range(int(t[0]), int(t[1]) + 1) for t in size_classes], key=lambda r: (r.start, r.stop))
            schedule = fumbblapi.get__tournament_schedule(tournamentId)
            teams = len(srschedule.teams(schedule))
            for size_class in size_classes:
                if teams in size_class:
                    class_parts[3] = f'{size_class.start}-{size_class.stop-1}'
            # we now have the tournament class
            class_ = '/'.join(class_parts)
            top_id = (None if class_.startswith('QU') else 0)
            row = [tournamentId, top_id, groupId, name, class_]
            if not class_.startswith('QU'):
                row.extend([title, slot, None, None])
            print(row)
            print()
            # add row to the tournament data
            new_rows.append(row)
            srdata.data["tournament"][row[TOURNAMENT]] = row
            bisect.insort(srdata.data["tournaments"], row)

## Refresh 'In Progress' Tournaments

In [10]:
for row in srdata.data["tournaments"]:
    if row[TOP_TOURNAMENT] or len(row) < ENTER_WEEKNR or row[ENTER_WEEKNR]:
        continue
    group_tournaments = fumbblapi.get__group_tournaments(row[GROUP])
    tournament = [t for t in group_tournaments if t["id"] == row[TOURNAMENT]][0]
    if tournament["status"] != 'Completed':
        continue
    schedule = fumbblapi.get__tournament_schedule(row[0])
    report_weeknr = row[ENTER_WEEKNR] = srschedule.report_weeknr(schedule)
    row[EXIT_WEEKNR] = report_weeknr + (78 if row[-4] else 52)
    print('Completed: ', row)
    if row[TITLE]:
        for row2 in srdata.data["tournaments"]:
            if len(row2) <= TITLE:
                continue
            if row2[TITLE] != row[TITLE]:
                continue
            if row2 == row:
                continue
            if row2[ENTER_WEEKNR] + 78 == row2[EXIT_WEEKNR] and row[ENTER_WEEKNR] <= row2[EXIT_WEEKNR]:
                print(f'Exit weeknr changed for the following tournament from {row2[EXIT_WEEKNR]} to {row[ENTER_WEEKNR]}:')
                row2[EXIT_WEEKNR] = row[ENTER_WEEKNR]
                print(row2)

## Save TOURNAMENTS.JSON

In [110]:
t = '[\n' + ',\n'.join(json.dumps(row) for row in srdata.data["tournaments"]) + '\n]'

In [111]:
with open(sr_data_path + '/tournaments.json', 'w') as f:
    f.write(t)

## Tournaments Table

In [18]:
weeknr_report_number = {w: r for r, w in enumerate(srtime.report_weeknrs(), 1)}

In [17]:
nameurlfs = 'https://fumbbl.com/p/group?op=view&at=1&group={group}&p=tournaments&show={tournament}&showallrounds=1'

In [75]:
rank_order = ('MA', 'QU', 'MI')
pending_rows_ = []
finished_rows_ = []
for row in srdata.data["tournaments"]:
    name = row[NAME]
    nameurl = nameurlfs.format(group=row[GROUP], tournament=row[TOURNAMENT])
    tname = f'[{name}|{nameurl}]'
    rank, level, format_, teams = row[CLASS].split('/')
    points = srdata.data['points'][row[CLASS]]
    first_slot_group = ' '
    enter_date, exit_date = ' ', ' '
    enter_report, exit_report = None, None
    if row[TOP_TOURNAMENT] is None:
        top_name = ''
        top_enter_weeknr = None
    elif row[TOP_TOURNAMENT] == 0:
        first_slot_group = row[FIRST_SLOT_GROUP]
        top_name = name
        top_enter_weeknr = row[ENTER_WEEKNR]
        if top_enter_weeknr:
            enter_date = srtime.weeknr_firstdate(top_enter_weeknr).isoformat()
            exit_date = srtime.weeknr_firstdate(row[EXIT_WEEKNR]).isoformat()
            enter_report = weeknr_report_number[top_enter_weeknr]
            exit_report = weeknr_report_number.get(row[EXIT_WEEKNR])
    else:
        top_row = srdata.data["tournament"][row[TOP_TOURNAMENT]]
        top_name = top_row[NAME]
        top_enter_weeknr = top_row[ENTER_WEEKNR]
    if enter_report:
        tenter_date = f'[{enter_date}|SR-Report-{enter_report}]'
    else:
        tenter_date = enter_date
    if exit_report:
        texit_date = f'[{exit_date}|SR-Report-{exit_report}]'
    elif row[TOP_TOURNAMENT] == 0 and row[ENTER_WEEKNR] and row[EXIT_WEEKNR] and row[ENTER_WEEKNR] + 52 < row[EXIT_WEEKNR]:
        texit_date = f'({exit_date})'
    else:
        texit_date = exit_date
    if top_enter_weeknr:
        sort_values = [-top_enter_weeknr, (rank == 'MI'), top_name, (rank == 'QU'), name]
        rows = finished_rows_
    else:
        sort_values = [0, (rank == 'MI'), top_name, (rank == 'QU'), name]
        rows = pending_rows_
    table_values = [tname, rank, level, format_, teams, points, first_slot_group, tenter_date, texit_date]
    bisect.insort(rows, [sort_values, table_values])
pending_rows = [t[1] for t in pending_rows_]
finished_rows = [t[1] for t in finished_rows_]

In [38]:
pending_rows

[['[COS IX Q:1|https://fumbbl.com/p/group?op=view&at=1&group=7067&p=tournaments&show=45040&showallrounds=1]',
  'QU',
  '1',
  'E',
  '13-16',
  '7 :: 15-30-60-100',
  ' ',
  ' ',
  ' '],
 ['[COS IX Q:2|https://fumbbl.com/p/group?op=view&at=1&group=7067&p=tournaments&show=45041&showallrounds=1]',
  'QU',
  '1',
  'E',
  '13-16',
  '7 :: 15-30-60-100',
  ' ',
  ' ',
  ' '],
 ['[COS IX Q:3|https://fumbbl.com/p/group?op=view&at=1&group=7067&p=tournaments&show=45042&showallrounds=1]',
  'QU',
  '1',
  'E',
  '13-16',
  '7 :: 15-30-60-100',
  ' ',
  ' ',
  ' '],
 ['[COS IX Q:4|https://fumbbl.com/p/group?op=view&at=1&group=7067&p=tournaments&show=45043&showallrounds=1]',
  'QU',
  '1',
  'E',
  '13-16',
  '7 :: 15-30-60-100',
  ' ',
  ' ',
  ' '],
 ['[COS IX Q:5|https://fumbbl.com/p/group?op=view&at=1&group=7067&p=tournaments&show=45044&showallrounds=1]',
  'QU',
  '1',
  'E',
  '13-16',
  '7 :: 15-30-60-100',
  ' ',
  ' ',
  ' '],
 ['[COS IX Q:6|https://fumbbl.com/p/group?op=view&at=1&group

In [68]:
pending = srpages.table(
      [row[:-2] for row in pending_rows],
      align='LCCCCLC',
      header=['Name', 'Rank', 'Level', 'Format', 'Teams', 'Points', 'First Slot Group']
)

In [76]:
finished = srpages.table(
      [row for row in finished_rows],
      align='LCCCCLCRR',
      header=['Name', 'Rnk', 'Lvl', 'Fmt', 'Tms', 'Points', 'FSG', 'Enter Date', 'Exit Date']
)

In [109]:
nonelim_rounds = {}
for j, row in enumerate(srdata.data["tournaments"][i:]):
    #print(row[NAME])
    #print(srtournament.format_(row[GROUP], row[TOURNAMENT]))
    schedule = fumbblapi.get__tournament_schedule(row[TOURNAMENT])
    rounds = srschedule.rounds(schedule)
    teams = len(srschedule.teams(schedule))
    #print(teams)
    class_parts = row[CLASS].split('/')
    if not '-' in class_parts[-1]:
        class_parts[-1] = f'{class_parts[-1]}-{class_parts[-1]}'
    teams_range_values = [int(a) for a in class_parts[-1].split('-')]
    teams_range_values[-1] += 1
    teams_range = range(*teams_range_values)
    if not teams in teams_range:
        print(f'{row[TOURNAMENT]} ({row[NAME]}) teams error!')
        break
    if class_parts[-2] == 'E':
        class_parts[-1] = str(teams)
    else:
        nonelim_rounds.setdefault(row[CLASS], []).append(rounds)
        class_parts[-1] = str(rounds)
    row[CLASS] = '/'.join(class_parts)

In [90]:
print(teams, teams_range)

12 range(13, 17)


In [107]:
i, j

(158, 0)

In [108]:
i += j + 1

In [102]:
i = 154

In [93]:
print(row)

[10515, 0, 2294, 'RRR XXI', 'MI/1/E/13-16', '', 'R', 163, 215]


In [94]:
schedule

[{'created': None,
  'modified': None,
  'position': 8,
  'result': {'id': 1026760,
   'replayId': 0,
   'teams': [{'id': 222309, 'score': 4}, {'id': 223993, 'score': 3}],
   'winner': '222309'},
  'round': 1,
  'teams': [{'id': 222309, 'name': 'NES Tribute'},
   {'id': 223993, 'name': 'Humbleton Hammers'}]},
 {'created': None,
  'modified': None,
  'position': 9,
  'result': {'id': 1028803,
   'replayId': 0,
   'teams': [{'id': 223243, 'score': 1}, {'id': 224355, 'score': 2}],
   'winner': '223243'},
  'round': 1,
  'teams': [{'id': 223243, 'name': 'Celtic Pagans'},
   {'id': 224355, 'name': 'Apathy Allstars'}]},
 {'created': None,
  'modified': None,
  'position': 11,
  'result': {'id': 1027180,
   'replayId': 0,
   'teams': [{'id': 223956, 'score': 0}, {'id': 223725, 'score': 1}],
   'winner': '223956'},
  'round': 1,
  'teams': [{'id': 223956, 'name': 'Tombstone Turmoil'},
   {'id': 223725, 'name': 'Skreetches'}]},
 {'created': None,
  'modified': None,
  'position': 12,
  'result'

In [112]:
nonelim_rounds

{'MI/1/N/2-6': [5],
 'MI/1/N/7-24': [4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4],
 'QU/1/N/7-24': [4, 4],
 'QU/2/N/7-24': [3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 3]}

In [114]:
has_fillers = set()
for j, row in enumerate(srdata.data["tournaments"]):
    #print(row[NAME])
    #print(srtournament.format_(row[GROUP], row[TOURNAMENT]))
    schedule = fumbblapi.get__tournament_schedule(row[TOURNAMENT])
    if srschedule.has_filler(schedule):
        has_fillers.add(row[TOURNAMENT])

In [116]:
min(has_fillers)

16919